## 회귀분석
### 보스턴 집값 예측
    회귀식 세우고 predict 값이랑 비교
    Accuracy는 predict 값이랑 test값과의 차의 제곱
    

In [ ]:
import torch
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from sklearn.datasets import load_boston
from sklearn.datasets import make_regression
import sklearn.linear_model as lm
import statsmodels.api as sm
import matplotlib.font_manager as fm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


# # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
# import matplotlib as mpl
# mpl.rcParams['axes.unicode_minus'] = False

# # GPU 설정 : 런타임 > 런타임 유형 변경 > GPU
# # GPU 사용 가능한지 확인
# device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device

In [ ]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 
# 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
# from google.colab import drive
# drive.mount('.playdata/ice/py_workspace/NLP/소설작가분류_NLP/data')
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# 특정파일을 가져오고 싶은 경우 다음과 같이 접근한다.
bst=load_boston()
dir(bst)
bst

In [ ]:
# 데이터 프레임 형태로 가공
bst_data=pd.DataFrame(data=bst.data,columns=bst.feature_names)
bst_target=pd.DataFrame(data=bst.target,columns=["House_Price_Mediv"]) 
bst_all=pd.concat([bst_data,bst_target],axis=1)

In [ ]:
# pairplot
print(bst_all.columns)
bst_all

In [ ]:
## 분포 그래프
bst_plot=sns.pairplot(bst_all[["House_Price_Mediv", "RM", "AGE", "CHAS"]])

In [ ]:
## Create Model
bst_rg = lm.LinearRegression(fit_intercept=True)

In [ ]:
## Create Traing set and Test set with Shuffling
bst_shuffled=bst_all.sample(frac=1).reset_index(drop=True)
bst_train=bst_shuffled.iloc[:int(len(bst_shuffled.index)*0.75)]
bst_test=bst_shuffled.iloc[int(len(bst_shuffled.index)*0.75):]

In [ ]:
## Train
rg_model=bst_rg.fit(bst_train.iloc[:,:-1],bst_train.iloc[:,-1])

In [ ]:
# 추정치
print(rg_model.coef_)
print(rg_model.intercept_)

In [ ]:
# Predict
bst_predict=rg_model.predict(bst_test.iloc[:,:-1])

In [ ]:
# 정확도 확인
# print(bst_test.iloc[:,-1],"\n 길이 : ",len(bst_test.iloc[:,-1]))
# print(bst_predict,"\n 길이 : ",len(bst_predict))
def residual(predict,test):
    diff_pred_test=np.array(predict)-np.array(test)
    sqrt_diff=np.square(diff_pred_test)   
    return np.sum(sqrt_diff)

residual(bst_predict,bst_test.iloc[:,-1])

In [ ]:
plt.plot(bst_predict)
plt.plot(bst_test.iloc[:,-1])
plt.show()

## 회귀분석
### 보스턴 집값 예측
    OLS 모델로 추정

In [ ]:
## OLS 모델링
bst_scale=StandardScaler().fit_transform(bst_all)
bst_scale=pd.DataFrame(data=bst_scale,index=bst_all.index,columns=bst_all.columns)
bst_train=bst_scale.iloc[:int(len(bst_scale.index)*0.75)]
bst_test=bst_scale.iloc[int(len(bst_scale.index)*0.75):]

bst_ols=sm.OLS.from_formula("House_Price_Mediv~NOX+RM+DIS+RAD+PTRATIO+LSTAT"
                            ,data=bst_train)

In [ ]:
# Training the Model and Summary one 
model=bst_ols.fit()
print(model.summary())

In [ ]:
# Predict
bst_predict=model.predict(bst_test.iloc[:,:-1])
bst_predict

In [ ]:
# 잔차 그래프
model.resid.plot(style="o")
plt.title("잔차 벡터")
plt.xlabel("데이터 번호")
plt.ylabel("잔차")
plt.show()

In [ ]:
# 잔차
print(model.resid.sum())

In [ ]:
residual(bst_predict,bst_test.iloc[:,-1])

## 전처리 및 PCA 적용한 회귀분석
1. Scaling(Normalize)
2. Check Correlation & Multi Linear
3. Principle Component Analysis
4. Regression Analysis


In [ ]:
## 변수간 상관관계 및 다중공선성
cmap = sns.light_palette("darkgray", as_cmap=True)
sns.heatmap(bst_all.corr(), annot=True, cmap=cmap)
plt.show()
bst_all.corr()*100

In [ ]:
## 스케일링
bst_scale=StandardScaler().fit_transform(bst_all)
bst_scale=pd.DataFrame(data=bst_scale,index=bst_all.index,columns=bst_all.columns)
bst_scale

In [ ]:
# 주성분 분석
## 주성분을 몇개로 할지 결정. 이거는 사람이 보고 정해줘야 함
def min_pca():
    pca_sum=[]
    for i in range(1,len(bst_scale.columns)-1):
        pca = PCA(n_components=i)
        printcipalComponents = pca.fit_transform(bst_scale.iloc[:,:-1])
        principalDf = pd.DataFrame(data=printcipalComponents)
        pca_sum.append(round(sum(pca.explained_variance_ratio_),4))
    return pca_sum
min_pca()
#[0,0.4722,0.5843,0.677,0.7448,0.81,0.8623,0.8994,0.9302,0.9513,0.9682,0.982,0.9948,1.0]


In [ ]:
# 주성분 5개까지 뽑기
n=5
pca = PCA(n_components=n)
col_name=['pc'+str(i) for i in range(1,n+1)]
printcipalComponents = pca.fit_transform(bst_scale.iloc[:,:-1])
principalDf = pd.DataFrame(data=printcipalComponents,
                           columns = col_name)
principalDf
pca_bst=pd.concat([principalDf.iloc[:len(bst_train),:],bst_train.iloc[:,-1]],axis=1)
pca_bst

In [ ]:
# Create Traing set and Test set
bst_train=pca_bst.iloc[:int(len(pca_bst.index)*0.75)]
bst_test=pca_bst.iloc[int(len(pca_bst.index)*0.75):]
bst_test

In [ ]:
## 회귀분석
pca_bst_ols=sm.OLS.from_formula("House_Price_Mediv ~ pc1+pc2+pc3+pc4+pc5"
                                ,data=bst_train)
pca_bst_model=pca_bst_ols.fit()

In [ ]:
print(pca_bst_model.summary())

In [ ]:
# Predict# Predict
bst_predict=pca_bst_model.predict(bst_test.iloc[:,:-1])
bst_predict

In [ ]:
# 잔차
pca_bst_model.resid.sum()

In [ ]:
print("[bst_predict]\n",bst_predict,"\n\n [bst_test] \n",bst_test.iloc[:,-1])
print("========================================================")
residual(bst_predict,bst_test.iloc[:,-1])